In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
import json 
import numpy as np
import os
import re
import pycoq.parse
import pycoq.common
import pycoq.serapi
from pycoq.sexp import  token_generator, sexp_, sexp
from pycoq.parse import ObjList
from typing import List

In [3]:
# reference implementation
small_input = [r'()',r'(())', r'(()  ()  )', r'(abb(()d)bcc)' ,r'(abb"b\"c\\c"())']
small_output = list(map(sexp, small_input))
# expected output: [[], [[]], [[], []], ['abb', [[], 'd'], 'bcc'], ['abb', '"b\\"c\\\\c"', []]]
with open('../tests/sexp.txt','w') as f:
    for line in small_input:
        f.write(line + '\n')

In [4]:
def query_responses(source_directory):
    res = []
    for source in pycoq.common.find_files(source_directory, r'.*' + pycoq.common._DEFAULT_SERAPI_LOGEXT + '$'):
        with open(source) as f:
            s = json.load(f)
            for line in s['response']:
                match = pycoq.serapi.ANSWER_PATTERN_OBJLIST.match(line)
                if match:
                    res.append(match.group(2).strip())
    return res

def readlines(fname: str) -> List[str]:
    res = []
    with open(fname,'r') as f:
        for line in f.readlines():
            res.append(line.strip())
    return res
        
def writelines(lines, fname):
    with open(fname,'w') as f:
        for line in lines:
            f.write(line +'\n')

In [5]:
# 600Mb takes 2.5 seconds to read lines into memory from file

In [6]:
res1 = readlines('../tests/bignums-query-responses.txt')[3000:3001]
sum(len(line) for line in res1)
writelines(res1, '../tests/0Mb.txt')

In [45]:
res1 = readlines('../tests/bignums-query-responses.txt')[3000:3330]

writelines(res1, '../tests/1Mb.txt')
sum(len(line) for line in res1)

1003938

In [46]:
res1 = readlines('../tests/bignums-query-responses.txt')[3000:4650]
writelines(res1, '../tests/10Mb.txt')
sum(len(line) for line in res1)

10337142

In [9]:
res1 = readlines('../tests/bignums-query-responses.txt')[3000:6300]
writelines(res1, '../tests/100Mb.txt')
sum(len(line) for line in res1)

108844443

In [10]:
res = readlines('../tests/0Mb.txt')
%time a = list(len(list(token_generator(line))) for line in res)

CPU times: user 541 µs, sys: 153 µs, total: 694 µs
Wall time: 698 µs


In [11]:
res = readlines('../tests/1Mb.txt')
%time a = list(len(list(token_generator(line))) for line in res)

CPU times: user 469 ms, sys: 0 ns, total: 469 ms
Wall time: 468 ms


In [12]:
res = readlines('../tests/10Mb.txt')
%time a = sum(len(list(token_generator(line))) for line in res)
a

CPU times: user 4.55 s, sys: 0 ns, total: 4.55 s
Wall time: 4.55 s


4505673

In [129]:
res = readlines('../tests/100Mb.txt')
%time a = sum(len(list(token_generator(line))) for line in res)
a

CPU times: user 44.9 s, sys: 430 ms, total: 45.3 s
Wall time: 45.3 s


45896337

In [47]:
res = readlines('../tests/10Mb.txt')

In [48]:
import functools
def depth(root):
    if isinstance(root, str):
        return 1
    else:
        return 1 +  functools.reduce(max,(depth(item) for item in root),0)

In [49]:
%time writelines([str(depth(sexp_(token_generator(line)))) for line in res], '../tests/10Mb.depths.py')

CPU times: user 8.18 s, sys: 37 µs, total: 8.18 s
Wall time: 8.19 s


In [6]:
%time tok_gens = [token_generator(line) for line in r]
%time ss = [sexp_(t) for t in tok_gens]
%time os = [ObjList.from_sexp(s) for s in ss]

CPU times: user 2.66 s, sys: 36 ms, total: 2.7 s
Wall time: 2.7 s
CPU times: user 13 s, sys: 207 ms, total: 13.2 s
Wall time: 13.2 s
CPU times: user 486 ms, sys: 805 µs, total: 487 ms
Wall time: 487 ms


In [7]:
import sexpdata

In [8]:
%time ss2 = [sexpdata.loads(line) for line in r]


CPU times: user 15.9 s, sys: 404 ms, total: 16.3 s
Wall time: 16.3 s


In [9]:
import pycoq.lark_serapi

In [10]:
%time ss3 = [pycoq.lark_serapi.sexp_parser_mach.parse(line) for line in r]

CPU times: user 1min 12s, sys: 1.07 s, total: 1min 13s
Wall time: 1min 13s


[[], [[]], [[], []], ['abb', [], 'bcc'], ['abb', '"b\\"c\\\\c"', []]]